In [1]:
import pandas as pd
import gensim
from Word2VecUtility3 import Word2VecUtility3
from __future__ import division
from sklearn.cross_validation import train_test_split
import pickle
import numpy as np
from gensim.models import word2vec
import logging
import nltk.data

c:\users\james\anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
c:\users\james\anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('AFF_Evenly_Sampled.csv', encoding='utf-8')
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164000 entries, 0 to 163999
Data columns (total 2 columns):
Score    164000 non-null int64
Text     164000 non-null object
dtypes: int64(1), object(1)
memory usage: 2.5+ MB
None
   Score                                               Text
0      1  I like this brand. I didn't realize I was orde...
1      1  Being my wife is a licensed cosmetologist and ...
2      1  If you are looking for an upgrade from the sta...
3      1  I am so allergic to too many artificial sweete...
4      1  I have not been able to find this locally and ...


# Part 1: Train Word2vec Model on Corpora

In [3]:
#nltk.download()
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [4]:
review_sents = []
print("Cleaning and parsing the reviews...\n")
for i in range( 0, len(data["Text"])):
    # sent_reviews += Word2VecUtility.review_to_sentences(data["text"][i], tokenizer)
    review_sents += Word2VecUtility3.review_to_sentences(data.iloc[i]["Text"], tokenizer)

Cleaning and parsing the reviews...



c:\users\james\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
c:\users\james\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.....'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
c:\users\james\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.amazon.com/gp/product/B004HYXZB6/ref=cm_cr_rev_prod_title" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
c:\users\james\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'..

In [5]:
# review_sents = pickle.load(open('review_sents_1859888.pkl', 'rb'))
print(len(review_sents))
print(review_sents[:5])

843499
[['i', 'like', 'this', 'brand'], ['i', 'didn', 't', 'realize', 'i', 'was', 'ordering', 'extra', 'bold'], ['to', 'me', 'these', 't', 'cups', 'are', 'bitter'], ['since', 'i', 'can', 't', 'control', 'the', 'strength', 'i', 'make', 'a', 'small', 'cup', 'and', 'toss'], ['then', 'i', 'make', 'a', 'large', 'cup', 'to', 'drink']]


In [6]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)

print("Training model...")
model = word2vec.Word2Vec(review_sents, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "word2vecAFF"
model.save(model_name)

Training model...


In [7]:
model = word2vec.Word2Vec.load("word2vecAFF")

In [8]:
model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [9]:
model.wv['computer']  # numpy vector of a word

array([  6.57762066e-02,  -8.39834809e-02,   8.17066617e-03,
         1.04141282e-02,  -2.66474462e-03,  -2.27323528e-02,
        -2.14354713e-02,  -3.53646791e-03,  -4.52830121e-02,
         9.52791423e-02,  -1.06150575e-01,   7.16838837e-02,
        -5.28025031e-02,   9.54057649e-02,  -9.97108966e-03,
        -4.38281223e-02,  -2.80837640e-02,   8.40951204e-02,
         2.82229371e-02,  -4.01410945e-02,   1.00977402e-02,
        -4.42988314e-02,   4.73538451e-02,  -1.36963530e-02,
         4.46166731e-02,   6.49208650e-02,  -3.77086774e-02,
         7.24832341e-02,   4.36785296e-02,  -9.58878919e-02,
         4.29454399e-03,   1.80076547e-02,  -9.95138753e-03,
         5.35431653e-02,  -5.14382089e-04,  -2.79666297e-03,
         8.21596086e-02,  -2.82009020e-02,  -1.92336785e-03,
        -3.73922028e-02,   4.43739966e-02,   2.77878605e-02,
        -6.89840019e-02,  -5.17202402e-03,   2.13419367e-02,
        -4.61289883e-02,   9.91263986e-02,  -8.44704267e-03,
        -5.81067912e-02,

# Part 2: Creating Word Indices

In [10]:
from __future__ import print_function
import os
import sys
import numpy as np
import pandas as pd
import pickle
import re
import csv
import codecs
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import gensim
from Word2VecUtility3 import Word2VecUtility3

BASE_DIR = 'C:/Users/James/Desktop/NLP Summer/'
#EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin'
#GLOVE_DIR = BASE_DIR + '/glove.42B.300d/'
TRAIN_DATA_FILE = BASE_DIR + '/AFF/AFF_Evenly_Sampled.csv'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

In [11]:
# second, prepare text samples and their labels
print('Processing text dataset')

# The function "text_to_wordlist" is from
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text

# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

Processing text dataset


In [12]:
#create arrays for text data and labels

texts = [] 
labels = []
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts.append(text_to_wordlist(values[1]))
        labels.append(int(values[0]))
print('Found %s texts in train.csv' % len(texts))

#store and load using pickle
pickle.dump((texts, labels), open("texts_labels_index.pkl", 'wb'))
(texts, labels) = pickle.load(open("texts_labels_index.pkl", 'rb'))


tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

Found 164000 texts in train.csv
Found 69963 unique tokens
Shape of data tensor: (164000, 1000)
Shape of label tensor: (164000,)


Word2vec model

In [13]:
type(model.wv.syn0)

numpy.ndarray

In [14]:
model.wv.syn0.shape

(8839, 300)

In [15]:
#index2word_set = set(model.wv.index2word)

In [16]:
## prepare embeddings
########################################
print('Preparing embedding matrix')

nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix_w2v = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in model.wv.vocab:
        embedding_matrix_w2v[i] = model.wv.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix_w2v, axis=1) == 0))


Preparing embedding matrix
Null word embeddings: 61133


In [17]:
embedding_layer_w2v = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix_w2v],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)

In [18]:
num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25
act = 'relu'

In [19]:
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer_w2v(sequence_input)
x = lstm_layer(embedded_sequences)
x = Dropout(rate_drop_dense)(x)
x = BatchNormalization()(x)
x = Dense(num_dense, activation=act)(x)
x = Dropout(rate_drop_dense)(x)
x = BatchNormalization()(x)

preds = Dense(1, activation='sigmoid')(x)

In [20]:
## train the model
#from sklearn.cross_validation import StratifiedKFold


model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         20989200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 233)               497688    
_________________________________________________________________
dropout_1 (Dropout)          (None, 233)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 233)               932       
_________________________________________________________________
dense_1 (Dense)              (None, 124)               29016     
_________________________________________________________________
dropout_2 (Dropout)          (None, 124)               0         
__________

In [21]:
# Fit the model
model.fit(x_train, y_train, epochs=2, batch_size=128, verbose=2, validation_split =1/12.)

Train on 120266 samples, validate on 10934 samples
Epoch 1/2
1027s - loss: 0.4320 - acc: 0.8006 - val_loss: 0.3097 - val_acc: 0.8748
Epoch 2/2
1054s - loss: 0.4244 - acc: 0.7990 - val_loss: 0.3789 - val_acc: 0.8349


In [22]:
scores = model.evaluate(x_val, y_val, verbose=0)
print("Test Score: %.2f%%" % (scores[0]*100))
print("Accuracy: %.2f%%" % (scores[1]*100))

Test Score: 37.19%
Accuracy: 84.26%


In [23]:
# Fit the model
model.fit(x_train, y_train, epochs=2, batch_size=128, verbose=2)

Epoch 1/2
1140s - loss: 0.3187 - acc: 0.8646
Epoch 2/2
1111s - loss: 0.2692 - acc: 0.8893


In [24]:
scores = model.evaluate(x_val, y_val, verbose=0)
print("Test Score: %.2f%%" % (scores[0]*100))
print("Accuracy: %.2f%%" % (scores[1]*100))

Test Score: 23.03%
Accuracy: 90.47%
